In [1]:
from metatrader.utils.factory import MetatraderFactory
metatrader = MetatraderFactory.get_metatrader()
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import pytz

mt5 = metatrader.connect()

symbol = 'SP500'
timeframe = mt5.TIMEFRAME_M5

trade_volume = 10
commission = 0.5



Connected to Metatrader successfully.


In [2]:
bars = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, 0,500))
bars['time'] = pd.to_datetime(bars['time'], unit='s')
bars['trade_volume'] = trade_volume
bars

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume
0,2023-10-23 05:00:00,4234.1,4234.7,4233.7,4234.0,37,3,0,10
1,2023-10-23 05:05:00,4234.2,4234.5,4233.4,4233.6,59,2,0,10
2,2023-10-23 05:10:00,4233.6,4233.6,4232.2,4232.2,32,1,0,10
3,2023-10-23 05:15:00,4232.4,4232.6,4231.7,4232.2,67,1,0,10
4,2023-10-23 05:20:00,4232.2,4232.5,4230.4,4230.7,102,1,0,10
...,...,...,...,...,...,...,...,...,...
495,2023-10-24 23:15:00,4250.8,4252.9,4250.1,4250.1,312,2,0,10
496,2023-10-24 23:20:00,4250.3,4251.1,4248.4,4251.1,228,1,0,10
497,2023-10-24 23:25:00,4250.9,4251.1,4246.4,4247.9,203,2,0,10
498,2023-10-24 23:30:00,4247.9,4249.9,4247.6,4249.6,185,2,0,10


In [3]:
bars['sma_100'] = bars['close'].ewm(span=20).mean()
bars['sma_10'] = bars['close'].ewm(span=10).mean()
bars['hour'] = bars['time'].dt.hour
bars

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,sma_100,sma_10,hour
0,2023-10-23 05:00:00,4234.1,4234.7,4233.7,4234.0,37,3,0,10,4234.000000,4234.000000,5
1,2023-10-23 05:05:00,4234.2,4234.5,4233.4,4233.6,59,2,0,10,4233.790000,4233.780000,5
2,2023-10-23 05:10:00,4233.6,4233.6,4232.2,4232.2,32,1,0,10,4233.206162,4233.144850,5
3,2023-10-23 05:15:00,4232.4,4232.6,4231.7,4232.2,67,1,0,10,4232.915698,4232.833564,5
4,2023-10-23 05:20:00,4232.2,4232.5,4230.4,4230.7,102,1,0,10,4232.379740,4232.221076,5
...,...,...,...,...,...,...,...,...,...,...,...,...
495,2023-10-24 23:15:00,4250.8,4252.9,4250.1,4250.1,312,2,0,10,4248.449740,4250.450908,23
496,2023-10-24 23:20:00,4250.3,4251.1,4248.4,4251.1,228,1,0,10,4248.702146,4250.568925,23
497,2023-10-24 23:25:00,4250.9,4251.1,4246.4,4247.9,203,2,0,10,4248.625751,4250.083666,23
498,2023-10-24 23:30:00,4247.9,4249.9,4247.6,4249.6,185,2,0,10,4248.718536,4249.995727,23


In [4]:
bars2 = bars.dropna().copy()

def get_signal(x):

    if 9 <= x['hour'] <= 22:

        if x['sma_10'] > x['sma_100']:
            return 1
        elif x['sma_10'] < x['sma_100']:
            return -1

    return 0

bars2['signal'] = bars2.apply(get_signal, axis=1)
bars2['prev_price'] = bars2['close'].shift(1)
bars2['price_change'] = bars2['close'] - bars2['prev_price']

bars2['signal_change'] = (bars2['signal'] - bars2['signal'].shift(1)).abs()
bars2['commission'] = bars2.apply(lambda x: commission * x['signal_change'] * x['trade_volume'], axis=1)
bars2

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,sma_100,sma_10,hour,signal,prev_price,price_change,signal_change,commission
0,2023-10-23 05:00:00,4234.1,4234.7,4233.7,4234.0,37,3,0,10,4234.000000,4234.000000,5,0,NaN,NaN,NaN,NaN
1,2023-10-23 05:05:00,4234.2,4234.5,4233.4,4233.6,59,2,0,10,4233.790000,4233.780000,5,0,4234.0,-0.4,0.0,0.0
2,2023-10-23 05:10:00,4233.6,4233.6,4232.2,4232.2,32,1,0,10,4233.206162,4233.144850,5,0,4233.6,-1.4,0.0,0.0
3,2023-10-23 05:15:00,4232.4,4232.6,4231.7,4232.2,67,1,0,10,4232.915698,4232.833564,5,0,4232.2,0.0,0.0,0.0
4,2023-10-23 05:20:00,4232.2,4232.5,4230.4,4230.7,102,1,0,10,4232.379740,4232.221076,5,0,4232.2,-1.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2023-10-24 23:15:00,4250.8,4252.9,4250.1,4250.1,312,2,0,10,4248.449740,4250.450908,23,0,4250.6,-0.5,0.0,0.0
496,2023-10-24 23:20:00,4250.3,4251.1,4248.4,4251.1,228,1,0,10,4248.702146,4250.568925,23,0,4250.1,1.0,0.0,0.0
497,2023-10-24 23:25:00,4250.9,4251.1,4246.4,4247.9,203,2,0,10,4248.625751,4250.083666,23,0,4251.1,-3.2,0.0,0.0
498,2023-10-24 23:30:00,4247.9,4249.9,4247.6,4249.6,185,2,0,10,4248.718536,4249.995727,23,0,4247.9,1.7,0.0,0.0


In [5]:
bars3 = bars2.dropna().copy()
bars3['profit'] = bars3['signal'] * bars3['price_change'] * bars3['trade_volume']
bars3['gross_profit'] = bars3['profit'].cumsum()
bars3['net_profit'] = bars3['gross_profit'] - bars3['commission'].cumsum()


bars3

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,sma_100,sma_10,hour,signal,prev_price,price_change,signal_change,commission,profit,gross_profit,net_profit
1,2023-10-23 05:05:00,4234.2,4234.5,4233.4,4233.6,59,2,0,10,4233.790000,4233.780000,5,0,4234.0,-0.4,0.0,0.0,-0.0,-0.0,-0.0
2,2023-10-23 05:10:00,4233.6,4233.6,4232.2,4232.2,32,1,0,10,4233.206162,4233.144850,5,0,4233.6,-1.4,0.0,0.0,-0.0,-0.0,-0.0
3,2023-10-23 05:15:00,4232.4,4232.6,4231.7,4232.2,67,1,0,10,4232.915698,4232.833564,5,0,4232.2,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-10-23 05:20:00,4232.2,4232.5,4230.4,4230.7,102,1,0,10,4232.379740,4232.221076,5,0,4232.2,-1.5,0.0,0.0,-0.0,0.0,0.0
5,2023-10-23 05:25:00,4230.4,4230.7,4228.7,4229.0,49,3,0,10,4231.666769,4231.384451,5,0,4230.7,-1.7,0.0,0.0,-0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2023-10-24 23:15:00,4250.8,4252.9,4250.1,4250.1,312,2,0,10,4248.449740,4250.450908,23,0,4250.6,-0.5,0.0,0.0,-0.0,1073.0,943.0
496,2023-10-24 23:20:00,4250.3,4251.1,4248.4,4251.1,228,1,0,10,4248.702146,4250.568925,23,0,4250.1,1.0,0.0,0.0,0.0,1073.0,943.0
497,2023-10-24 23:25:00,4250.9,4251.1,4246.4,4247.9,203,2,0,10,4248.625751,4250.083666,23,0,4251.1,-3.2,0.0,0.0,-0.0,1073.0,943.0
498,2023-10-24 23:30:00,4247.9,4249.9,4247.6,4249.6,185,2,0,10,4248.718536,4249.995727,23,0,4247.9,1.7,0.0,0.0,0.0,1073.0,943.0


In [6]:
px.line(bars3, 'time', ['gross_profit', 'net_profit'], title="SP500 NY session Backtest EMA Trendfollowing")


c:\Users\praveenjaisankar\AppData\Local\pypoetry\Cache\virtualenvs\metatrader-Cdlomroj-py3.10\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [7]:

fig = go.Figure(data=[go.Candlestick(x=bars3['time'],
                open=bars3['open'],
                high=bars3['high'],
                low=bars3['low'],
                close=bars3['close'])])

fig.show()

c:\Users\praveenjaisankar\AppData\Local\pypoetry\Cache\virtualenvs\metatrader-Cdlomroj-py3.10\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

